## Scenario 2

Owing to security concerns, a later version of firmware has been rolled out that changes the API. For Reasons, the rollout of the new firmware could not be completed across the board: some routers have been upgraded, some haven't.

### Sequence of events

* New TutorRouter firmware is rolled out to some routers. The script can no longer log in to or get traffic data from _all_ the routers in the same way.
    * Observe that further firmware updates will cause headaches...
    * Observe that other scripts doing different tasks will have to be modified.

### Demonstrate

* **Inheritance** - DRYness.
  * Each device _is a_ `TutorRouter` with some commonalities, e.g. IP address; health check path is always the same.
  * All TutorRouters can output total ingress and total egress.
  * Different login methodologies.
* **Encapsulation** - contain attributes (properties and methods) together in Objects. Each device _has_...
  * **Properties**: IP address/Hostname, set of interfaces.
  * **Methods**: a _method_ of logging in, a _method_ for checking health, a _method_ of pulling interface stats.
* **Abstraction** - hide implementation details that users of your class don't need or want to deal with.
  * Hide the details of how login is achieved.
  * Return the data (e.g. interface throughput) in a format that's easy to consume.
* **Polymorphism** - allow all routers to be treated the same way transparently.
  * Although different "shapes", they are all routers with the same functions.
  * No special logic should be needed by the user for routers with different firmware.

### Set up the simulation

In [5]:
# Load the libraries
import os, sys
sys.path.insert(0, "../src")
from tutor_router.scenarios import Scenario, NotFoundError

# Set up the scenario
scenario = Scenario.scenario_2()

# Make these look a bit like `requests` calls.
request = scenario.request
get = scenario.get
post = scenario.post

# Get a list of host names, for convenience.
hosts = tuple(scenario.hosts())

## Modifying the Ops script

Let's see what happens when we run the script now...

In [6]:
import json

# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Authentication": f"TOKEN {auth_token}"})
    data = json.loads(response)
    total_throughput = data["eth0"]["up"] + data["eth0"]["down"]
    print(f"  Throughput: {total_throughput} bits per second")

Host TutorRouter-41 is not available
Host TutorRouter-42 is up
  Throughput: 374847 bits per second
Host TutorRouter-43 is up
  Failed to log in
Host TutorRouter-44 is up
  Failed to log in
Host TutorRouter-45 is up
  Throughput: 655570 bits per second
Host TutorRouter-46 is up
  Failed to log in
Host TutorRouter-47 is up
  Throughput: 1168167 bits per second
Host TutorRouter-48 is not available
Host TutorRouter-49 is not available
Host TutorRouter-50 is up
  Throughput: 1018071 bits per second
Host TutorRouter-51 is up
  Failed to log in
Host TutorRouter-52 is not available
Host TutorRouter-53 is not available
Host TutorRouter-54 is up
  Throughput: 865434 bits per second
Host TutorRouter-55 is not available
Host TutorRouter-56 is up
  Failed to log in
Host TutorRouter-57 is up
  Throughput: 1073972 bits per second
Host TutorRouter-58 is up
  Throughput: 841605 bits per second
Host TutorRouter-59 is not available
Host TutorRouter-60 is up
  Failed to log in


OK well some are still working and others are failing at login; those would be the upgraded routers.

When login fails, let's try the V2 method of logging in.

In [8]:
import json

# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        # OK so we must be on a V2 router. Try this method instead...
        status_code, response = post(hostname, "authenticate", headers={"token": "token123456"})
        if status_code != 200:
            print(f"  Failed to log in")
            continue
    print("  Logged in")
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Authentication": f"TOKEN {auth_token}"})
    data = json.loads(response)
    total_throughput = data["eth0"]["up"] + data["eth0"]["down"]
    print(f"  Throughput: {total_throughput} bits per second")

Host TutorRouter-41 is not available
Host TutorRouter-42 is up
  Logged in
  Throughput: 374847 bits per second
Host TutorRouter-43 is up


TypeError: 'dict' object is not callable

In [12]:
scenario._routers["TutorRouter-43"]._login_path()

'authenticate'

Cool, now we can get the interface stats.

In [7]:
# Iterate over all routers
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # Now get the interface stats.
    status_code, response = get(hostname, "interfaces")
    print(f"  interfaces: ({status_code}) {response}")

Host TutorRouter-1 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-2 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-3 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-4 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-5 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-6 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-7 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-8 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-9 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-10 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-11 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-12 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-13 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRouter-14 is up
  interfaces: (403) {"message": "Forbidden"}
Host TutorRoute

Ah, that function needs login. OK let's do that.

In [8]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    print(f"  log in: ({status_code}) {response}")

Host TutorRouter-1 is up
  log in: (200) d50948d3-839c-4a9c-921a-1041510a43e1
Host TutorRouter-2 is up
  log in: (200) 816bdfb6-4956-4ad7-97bd-3b884ff5593d
Host TutorRouter-3 is up
  log in: (200) ec48faa8-405c-4877-9a79-0806cfaf8a55
Host TutorRouter-4 is up
  log in: (200) 77916ae3-15d5-440c-b4d6-c325579b9763
Host TutorRouter-5 is up
  log in: (200) f2604621-21eb-49c1-a63e-20dc80d5b34f
Host TutorRouter-6 is up
  log in: (200) 808deec2-1bbe-44c1-970a-c6b03a28a71a
Host TutorRouter-7 is up
  log in: (200) 492b6368-273d-49c5-b0d1-e25258dc5d81
Host TutorRouter-8 is up
  log in: (200) 6d851b9f-37b0-4149-99be-2b88a9b2e7d9
Host TutorRouter-9 is up
  log in: (200) d17237b5-def3-4ce3-ab8c-7bc26b9d08f3
Host TutorRouter-10 is up
  log in: (200) a5b915e9-7117-40c0-bb0b-4981952baf91
Host TutorRouter-11 is up
  log in: (200) 51e5404b-edb8-4f8a-8566-614f5385e7f8
Host TutorRouter-12 is up
  log in: (200) 22158e75-4a6a-4ba8-a676-a593fd6310cc
Host TutorRouter-13 is up
  log in: (200) 8c75b67c-bbdd-40b5-

The response to login (again: in this simulation) is an auth token that we have to pass in the headers with every request to a protected endpoint.

Let's do that.

Also, let's not print out the auth token any more :-)

In [9]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    print(f"  Logged in")
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Authentication": f"TOKEN {auth_token}"})
    if status_code != 200:
        print(f"  Failed to get interface stats")
        continue
    print(f"  Interfaces: ({status_code}) {response}")

Host TutorRouter-1 is up
  Logged in
  interfaces: (200) {"eth0": {"up": 975799, "down": 817226}, "mgmt": {"up": 428990, "down": 174309}}
Host TutorRouter-2 is up
  Logged in
  interfaces: (200) {"eth0": {"up": 188473, "down": 219346}, "mgmt": {"up": 145543, "down": 500571}}
Host TutorRouter-3 is up
  Logged in
  interfaces: (200) {"eth0": {"up": 854882, "down": 161629}, "mgmt": {"up": 251209, "down": 192095}}
Host TutorRouter-4 is up
  Logged in
  interfaces: (200) {"eth0": {"up": 381415, "down": 392222}, "mgmt": {"up": 208751, "down": 543973}}
Host TutorRouter-5 is up
  Logged in
  interfaces: (200) {"eth0": {"up": 258783, "down": 90610}, "mgmt": {"up": 993900, "down": 520838}}
Host TutorRouter-6 is up
  Logged in
  interfaces: (200) {"eth0": {"up": 411358, "down": 8292}, "mgmt": {"up": 599212, "down": 536021}}
Host TutorRouter-7 is up
  Logged in
  interfaces: (200) {"eth0": {"up": 456018, "down": 158655}, "mgmt": {"up": 28408, "down": 753009}}
Host TutorRouter-8 is up
  Logged in
 

Noice! Now we can just add the "up" and "down" for `eth0`, and job done!

In [14]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    print(f"  Logged in")
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Authentication": f"TOKEN {auth_token}"})
    if status_code != 200:
        print(f"  Failed to get interface stats")
        continue
    total_throughput = response["eth0"]["up"] + response["eth0"]["down"]
    print(f"  Throughput: {total_throughput} bits per second")

Host TutorRouter-21 is up
  Logged in


TypeError: string indices must be integers

WAT?

Ah. Turns out the response isn't a Python object, it's a JSON string blob. Let's fix that.

In [15]:
import json

# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    auth_token = response

    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Authentication": f"TOKEN {auth_token}"})
    data = json.loads(response)
    total_throughput = data["eth0"]["up"] + data["eth0"]["down"]
    print(f"  Throughput: {total_throughput} bits per second")

Host TutorRouter-21 is up
  Throughput: 1432472 bits per second
Host TutorRouter-22 is up
  Throughput: 1173016 bits per second
Host TutorRouter-23 is up
  Throughput: 991741 bits per second
Host TutorRouter-24 is up
  Throughput: 624717 bits per second
Host TutorRouter-25 is not available
Host TutorRouter-26 is up
  Throughput: 1154260 bits per second
Host TutorRouter-27 is not available
Host TutorRouter-28 is up
  Throughput: 1261948 bits per second
Host TutorRouter-29 is not available
Host TutorRouter-30 is not available
Host TutorRouter-31 is up
  Throughput: 1446865 bits per second
Host TutorRouter-32 is up
  Throughput: 1094135 bits per second
Host TutorRouter-33 is not available
Host TutorRouter-34 is up
  Throughput: 1326404 bits per second
Host TutorRouter-35 is not available
Host TutorRouter-36 is up
  Throughput: 1662675 bits per second
Host TutorRouter-37 is up
  Throughput: 608570 bits per second
Host TutorRouter-38 is up
  Throughput: 819639 bits per second
Host TutorRout

Job done!

But wait...